In [8]:
!pip install langchain_community langchain-openai openai

In [15]:
import os, json
from tenacity import retry, stop_after_attempt, wait_exponential
from datetime import datetime
from langchain_openai import OpenAI
from langchain import LLMChain, PromptTemplate
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.callbacks import get_openai_callback, StdOutCallbackHandler
from langchain.memory import ConversationBufferMemory

In [14]:
from google.colab import userdata
from openai import DefaultHttpxClient

client = OpenAI(
    base_url=userdata.get('baseurl'),
    api_key=userdata.get('apikey'),
    model='Qwen/Qwen3-235B-A22B',
    http_client=DefaultHttpxClient(verify=False),
    temperature=0.3
)

In [16]:
def quiz_fn(topic: str) -> str:
    tmpl = [
        "Ты эксперт в составлении квизов"
        "Тема: {topic}",
        "Составь вопрос квиза на заданную тему с 4 вариантами ответа."
        "Формат ответа: <q>str<\q>",
        "<ans1>str<\ans1>",
        "<ans2>str<\ans2>",
        "<ans3>str<\ans3>",
        "<ans4>str<\ans4>",
        "<right_ans>int<\right_ans>"
    ]
    tmpl = "\n".join(tmpl)
    prompt = PromptTemplate.from_template(tmpl)
    # Use RunnableSequence instead of LLMChain and invoke instead of run
    chain = prompt | client
    return chain.invoke({"topic": topic})

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=1, max=4))
def safe_quiz_fn(topic: str) -> str:
    return quiz_fn(topic)

def fallback_fn(topic: str) -> str:
    return "Извините, не удалось сгенерировать вопрос квиза после нескольких попыток."

# 4) Инструмент Reflection для самокритики
def reflection_fn(context: str) -> str:
    tmpl = (
        "Ты — эксперт по рефлексии результатов агента. "
        "Проанализируй следующий вывод и предложи улучшения:\n\n{context}"
    )
    prompt = PromptTemplate.from_template(tmpl)
    chain = prompt | client
    return chain.invoke({"context": context})

# 5) Контент-полиси фильтр (простой пример)
def policy_filter(text: str) -> str:
    # ловим email как PII
    if re.search(r"[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}", text):
        return "[Отфильтровано: обнаружено PII]"
    return text

# 6) Регистрируем инструменты
tools = [
    Tool.from_function(safe_quiz_fn,   name="quiz",     description="Генерация вопроса квиза"),
    Tool.from_function(reflection_fn,  name="reflect",  description="Самокритика/рефлексия"),
    Tool.from_function(fallback_fn,    name="fallback", description="Фоллбэк инструмент"),
]

In [17]:
memory = ConversationBufferMemory() # Память

callbacks = [StdOutCallbackHandler()] # Callback-handlers: вывод в stdout + сбор метрик

/tmp/ipython-input-17-2055361212.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory() # Память


In [23]:
agent = initialize_agent(
    tools,
    client,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    memory=memory,
    callbacks=callbacks,
    verbose=True,
    max_iterations=10,
    early_stopping_method="force",
    return_intermediate_steps=True,
    handle_parsing_errors=True,
    agent_kwargs={
        "prefix": (
            "Ты — генератор квизов. Твоя единственная работа — сгенерировать 5 вопросов с помощью quiz() "
            "Тема квиза — это {input}."
        ),
        "suffix": (
            "Выполни quiz()"
            "Если не удалось получить ответ от quiz() выполни fallback()."
            "Если quiz() вернул ответ вызови reflect()."
            "Повтори эти действия 5 раз"
            "Собери все ответы quiz() в один python список без изменений"
          )
    }
)

def main(topic_name):
    # собираем метрики по токенам и стоимости
    with get_openai_callback() as cb:
        result = agent({"input": topic_name})
    # сохраняем лог
    log = {
        "timestamp": datetime.utcnow().isoformat(),
        "input": topic_name,
        "output": result["output"],
        "metrics": {
            "total_tokens": cb.total_tokens,
            "prompt_tokens": cb.prompt_tokens,
            "completion_tokens": cb.completion_tokens,
            "total_cost_usd": cb.total_cost
        }
    }
    with open("quiz_run_log.json", "a", encoding="utf-8") as f:
        f.write(json.dumps(log, ensure_ascii=False) + "\n")
    return result

In [24]:
res = main('архитектуры llm агентов')



> Entering new AgentExecutor chain...
Выведи список в финальном ответе

Question: Сгенерируй 5 вопросов квиза по теме архитектуры llm агентов
Thought: I need to generate 5 quiz questions about LLM agent architectures. I'll use the quiz() function for each question and collect them in a list.
Action: quiz
Action Input: Архитектуры LLM агентов
Observation: Требования:1. Вопрос должен быть на знание терминологии и понимание архитектур LLM агентов2. Варианты ответов должны быть схожи по структуре и длине3. Один правильный ответ, три отвлекающих4. Отвлекающие ответы должны быть реалистичными и соответствовать теме квиза5. Вопрос и ответы должны быть на русском языке
Пример:
<q>Какой из следующих компонентов обычно отвечает за хранение и обработку памяти в архитектуре LLM-агента?</q>
<ans1>Модуль долгосрочной памяти</ans1>
<ans2>Интерфейс взаимодействия с пользователем</ans2>
<ans3>Система обработки естественного языка</ans3>
<ans4>Механизм планирования действий</ans4>
<right_ans>1</right_

/usr/local/lib/python3.11/dist-packages/langchain/memory/chat_memory.py:55: UserWarning: 'ConversationBufferMemory' got multiple output keys: dict_keys(['output', 'intermediate_steps']). The default 'output' key is being used. If this is not desired, please manually set 'output_key'.
  warnings.warn(


In [ ]:
res

{'input': 'архитектуры llm агентов',
 'history': '',
 'output': 'Agent stopped due to iteration limit or time limit.',
 'intermediate_steps': [(AgentAction(tool='quiz', tool_input='topic = "архитектуры llm агентов', log='Question: выполнить 5 вызовов функции quiz() и записать результаты в список\nThought: нужно выполнить 5 вызовов функции quiz() с темой архитектуры llm агентов и собрать результаты\nAction: quiz\nAction Input: topic = "архитектуры llm агентов"'),
   ' \n<q>Какой из следующих вариантов является ключевым преимуществом архитектуры Large Language Model (LLM) агентов?<\\q>\n<ans1>Повышенная сложность модели<\\ans1>\n<ans2>Улучшение точности прогнозирования<\\ans2>\n<ans3>Повышенная способность к обобщению и адаптации<\\ans3>\n<ans4>Снижение потребления вычислительных ресурсов<\\ans4>\n<right_ans>3<\\right_ans>'),
  (AgentAction(tool='append', tool_input='["Какой из следующих вариантов является ключевым преимуществом архитектуры Large Language Model (LLM) агентов?<br><ans1>По

In [1]:
!pip install openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.5 MB/s eta 0:00:00


In [ ]:
import asyncio
from agents.mcp import MCPServerSse
from agents.mcp.server import RunContextWrapper     # Обёртка контекста для run_context
from agents.agent import Agent                      # Класс Agent из SDK

async def main():
    # 1. Подключение к публичному демо-серверу MCP
    async with MCPServerSse(
        params={"url": "https://public-mcp-demo-oai.agent-ready.ai/mcp"},
        name="public-demo",
        cache_tools_list=True  # опционально: кэшировать результат list_tools()
    ) as server:

        # 2. Создаём необходимые объекты для list_tools()
        run_context = RunContextWrapper(context=None)
        agent = Agent(
            name="demo-agent",
            instructions="You are a test agent for listing MCP tools.",
            tools=[]            # дополнительные встроенные инструменты Agent SDK
        )

        # 3. Запрашиваем список доступных инструментов
        tools = await server.list_tools(run_context, agent)

        # 4. Выводим результат
        print("Доступные инструменты MCP-сервера:")
        for t in tools:
            print(f"- {t.name}: {t.description}")

# 5. Запуск
asyncio.run(main())